<a href="https://colab.research.google.com/github/erayyildiz/TurkishConversationBot/blob/master/TurkishConversation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install torch --upgrade
!pip install --force https://github.com/chengs/tqdm/archive/colab.zip

Requirement already up-to-date: torch in /usr/local/lib/python3.6/dist-packages (1.0.0)
     \ 512kB 572kB/s
  Running setup.py bdist_wheel for tqdm ... - done
  Stored in directory: /tmp/pip-ephem-wheel-cache-ea73c91c/wheels/41/18/ee/d5dd158441b27965855b1bbae03fa2d8a91fe645c01b419896
Successfully built tqdm
thinc 6.12.1 has requirement wrapt<1.11.0,>=1.10.0, but you'll have wrapt 1.11.1 which is incompatible.
spacy 2.0.18 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
pymc3 3.6 has requirement joblib<0.13.0, but you'll have joblib 0.13.1 which is incompatible.
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [0]:
from __future__ import unicode_literals, division
from io import open
import unicodedata
import string
import re
import random
import os.path
import pickle
from torch.utils.data import Dataset, DataLoader
import os
from tqdm import tqdm, tqdm_notebook
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

In [3]:
!ls

sample_data


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
create_subtitle_utterences=False
if create_subtitle_utterences:
    !wget http://opus.nlpl.eu/download.php?f=OpenSubtitles2018%2Fmono%2FOpenSubtitles2018.raw.tr.gz
    !gunzip 'download.php?f=OpenSubtitles2018%2Fmono%2FOpenSubtitles2018.raw.tr.gz'
    !mv 'download.php?f=OpenSubtitles2018%2Fmono%2FOpenSubtitles2018.raw.tr' subtitles.txt
    previous_line = ""
    with open("subtitles.dialogues.txt", "w") as w:
        with open("subtitles.txt", "r") as f:
            for i, line in enumerate(f):
                line = line.strip()
                if line.startswith("-"):
                    line = line.replace("- ", "")
                    line = line.replace("-", "")
                    if previous_line:
                        w.write("{}\t{}\n".format(previous_line, line))
                    previous_line = line
                else:
                    previous_line = ""
    !mkdir "gdrive/My Drive/chatbot"
    !cp subtitles.dialogues.txt "gdrive/My Drive/chatbot/"

In [0]:
import os
import re
import random
from nltk.tokenize import casual_tokenize

EOS = "<EOS>"
SOS = "<SOS>"
PAD = "<PAD>"
MAX_LEN = 25

def to_lower(txt):
    txt = txt.replace('İ', 'i')
    txt = txt.replace('Ğ', 'ğ')
    txt = txt.replace('Ü', 'ü')
    txt = txt.replace('Ş', 'ş')
    txt = txt.replace('Ç', 'ç')
    txt = txt.replace('Ö', 'ö')
    return txt.lower()

In [0]:
utterance_regex = re.compile(r"^.*[a-zA-Z0-9]+.*$")

subtitle_utterances = []

def add_dialogue(q, a, subtitle_utterances):
    q = to_lower(q)
    a = to_lower(a).strip()
    q_tokens = [token for token in casual_tokenize(q) if utterance_regex.match(token)]
    a_tokens = [token for token in casual_tokenize(a) if utterance_regex.match(token)]
    
    if (MAX_LEN - 2) > len(q_tokens) > 0 and (MAX_LEN - 3) > len(a_tokens) > 0:
        subtitle_utterances.append((q_tokens, a_tokens))
        added_utterances = len(subtitle_utterances)
        if added_utterances % 100000 == 0:
            print("{} dialogues are read so far.".format(added_utterances))


with open("gdrive/My Drive/chatbot/subtitles.dialogues.txt", "r") as f:
    for line in f:
        parses = line.split("\t")
        q = parses[0].strip()
        # if q.endswith("?") or any([question_word in q for question_word in question_words]):
        add_dialogue(q, parses[1], subtitle_utterances) 
        if len(subtitle_utterances) > 7000000:
            break

100000 dialogues are read so far.
200000 dialogues are read so far.
300000 dialogues are read so far.
400000 dialogues are read so far.
500000 dialogues are read so far.
600000 dialogues are read so far.
700000 dialogues are read so far.
800000 dialogues are read so far.
900000 dialogues are read so far.
1000000 dialogues are read so far.
1100000 dialogues are read so far.
1200000 dialogues are read so far.
1300000 dialogues are read so far.
1400000 dialogues are read so far.
1500000 dialogues are read so far.
1600000 dialogues are read so far.
1700000 dialogues are read so far.
1800000 dialogues are read so far.
1900000 dialogues are read so far.
2000000 dialogues are read so far.
2100000 dialogues are read so far.
2200000 dialogues are read so far.
2300000 dialogues are read so far.
2400000 dialogues are read so far.
2500000 dialogues are read so far.
2600000 dialogues are read so far.
2700000 dialogues are read so far.
2800000 dialogues are read so far.
2900000 dialogues are read so

In [0]:
word_counts_dic = {}
for utterance in subtitle_utterances:
    for word in utterance[0]:
        if word not in word_counts_dic:
            word_counts_dic[word] = 1
        else:
            word_counts_dic[word] += 1

min_word_count = 10
words = [PAD, SOS, EOS]
words += [w for w, c in word_counts_dic.items() if c >= min_word_count]

words = list(words)
int2word = words
word2int = {c:i for i,c in enumerate(words)}

In [0]:
def indexesFromSentence(sentence):
    return [word2int[word] for word in sentence if word in word2int]


def tensorFromSentence(sentence, max_len=MAX_LEN, add_start_tag=False):
    indexes = indexesFromSentence(sentence)
    if add_start_tag:
        indexes = [word2int[SOS]] + indexes 
    indexes.append(word2int[EOS])
    for i in range(len(indexes), max_len):
        indexes.append(word2int[PAD])
    return torch.tensor(indexes, dtype=torch.long, device=device)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(pair[0])
    target_tensor = tensorFromSentence(pair[1], add_start_tag=True)
    return input_tensor, target_tensor

In [0]:
class UtteranceDataset(Dataset):
    
    def __init__(self, utterances, words):
        self.utterances = utterances
        self.int2word = words
        self.word2int = {c:i for i,c in enumerate(words)}
        
        
    def __len__(self):
        return len(self.utterances)

    def __getitem__(self, index):
        utterance = self.utterances[index]
        return tensorsFromPair(utterance)
    
utterance_dataset = UtteranceDataset(subtitle_utterances, words)
        

In [0]:
data_loader = DataLoader(utterance_dataset, batch_size=64, shuffle=True)

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ENCODER_PATH = "gdrive/My Drive/chatbot/encoder.model"
DECODER_PATH = "gdrive/My Drive/chatbot/decoder.model"        
        
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        

    def forward(self, x):
        embedded = self.embedding(x)
        output = embedded
        hidden = self.init_hidden(x.size(0))
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)
    
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, hidden, y):
        output = self.embedding(y)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output))
        return output
    
    def predict(self, hidden, max_len=25):
        decoded_words = []
        decoder_input = torch.tensor([[word2int[SOS]]], device=device)
        input_embedding = self.embedding(decoder_input)
        for i in range(max_len):
            output, hidden = self.gru(input_embedding, hidden)
            output = self.softmax(self.out(output[0]))
            topv, topi = output.data.topk(1)
            if topi.item() == word2int[EOS]:
                break
            else:
                decoded_words.append(int2word[topi.item()])
            decoder_input = topi.squeeze().detach()
            decoder_input = torch.tensor([[decoder_input]], device=device)
            input_embedding = self.embedding(decoder_input)

        return decoded_words

    def init_hidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)
    


In [0]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    loss = 0
    encoder_outputs, encoder_hidden = encoder(input_tensor)
    decoder_outputs = decoder(encoder_hidden, target_tensor)
    decoder_outputs = decoder_outputs[:, :-1, :]
    decoder_outputs = decoder_outputs.contiguous().view(decoder_outputs.shape[0]*decoder_outputs.shape[1], -1)
    expected_outputs = target_tensor[:, 1:].contiguous().view(1, -1).squeeze()
    
    loss = criterion(decoder_outputs, expected_outputs)

    return loss

In [0]:
hidden_size = 512
encoder = EncoderRNN(len(words), hidden_size).to(device)
decoder = DecoderRNN(hidden_size, len(words)).to(device)

In [0]:
def evaluate(encoder, decoder, sentence):
    with torch.no_grad():
        if isinstance(sentence, str):
            tokens = casual_tokenize(to_lower(sentence))
        elif isinstance(sentence, list):
            tokens = sentence
        else:
            raise TypeError("Wrong Input Type: {}".format(type(sentence)))
        input_tensor = tensorFromSentence(tokens)
        input_length = input_tensor.size()[0]
        input_tensor = input_tensor.view(1, *input_tensor.size())
        encoder_outputs, encoder_hidden = encoder(input_tensor)
        predictions = decoder.predict(encoder_hidden)
        return ' '.join(predictions)

In [0]:
def evaluateRandomly(encoder, decoder, utterances, n=10):
    for i in range(n):
        pair = random.choice(utterances)
        output_sentence = evaluate(encoder, decoder, pair[0])
        yield '{} <=> {}'.format(' '.join(pair[0]), output_sentence)

In [0]:
def load_model_states(model, path):
    """Load a previously saved model states."""
    # original saved file with DataParallel
    state_dict = torch.load(path)
    model.load_state_dict(state_dict)
        
if os.path.isfile(ENCODER_PATH):
    print("Loading Encoder")
    load_model_states(encoder, ENCODER_PATH)
if os.path.isfile(DECODER_PATH):
    print("Loading Decoder")
    load_model_states(decoder, DECODER_PATH)

In [0]:
criterion = nn.CrossEntropyLoss()
encoder_optimizer = optim.Adam(encoder.parameters())
decoder_optimizer = optim.Adam(decoder.parameters())


print_loss_total = 0  

print_every = 500
save_every = 5000

encoder_optimizer.zero_grad()
decoder_optimizer.zero_grad()
loss = 0.0

for i, (input_tensor, target_tensor) in enumerate(tqdm(data_loader, desc='Training:')):
    loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)

    print_loss_total += loss.item()
    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()


    if i % print_every == 0:
        print_loss_avg = print_loss_total / print_every
        print_loss_total = 0
        print(' Loss: {}'.format(print_loss_avg))
    if i % save_every == 0:
        for message in evaluateRandomly(encoder, decoder, subtitle_utterances):
            print(message)
        torch.save(encoder.state_dict(), ENCODER_PATH)
        torch.save(decoder.state_dict(), DECODER_PATH)

In [0]:
from IPython.display import clear_output
import time


while True:
    clear_output()
    time.sleep(1)
    input_txt = input()
    output_words = evaluate(encoder, decoder, input_txt)
    output_sentence = ' '.join(output_words[:-1])
    print(" - {}".format(output_sentence))
    time.sleep(2)
    